# Assignment - RNN Sequence Classification - McCartney

In this assignment, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 187 data points (columns) for each measurement. This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T187 are the values of a measurement on the timeline (there are 187 data points, or columns, in a single measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T187**. 

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


# Important note:

Looks like the original length of the sequences were not the same in the data set. For example, one sequence (i.e., row) had 187 columns, while another one had 150 only. Therefore, the creators of the data set did a "zero padding". This means that they added zeros at end of each sequence to make them the same length. This way, all sequences (i.e., rows) were made to have 187 columns.

If you don't account for the zero-padding, you will create biased models. To account for this, you can use the following as your first layer of your models:<br>
`tf.keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs])`<br>
(Of course, you need to enter your own values for n_steps and n_inputs). After you add this layer, continue with your other layers as usual.

# Read and Prepare the Data

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
data = pd.read_csv("heartbeat.csv")

In [3]:
data.shape

(9026, 188)

In [4]:
data.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T179,T180,T181,T182,T183,T184,T185,T186,T187,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000,0.626,0.276,0.3250,0.4310,0.3900,0.3940,0.3580,0.3740,0.3620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y = data['Target']
x = data.drop('Target', axis=1)

In [6]:
# Split the data

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

### Data Transformation

In [7]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)

In [8]:
#Check the first 10 values of the train_y data set
train_y[0:10]

array([0, 2, 0, 0, 0, 0, 0, 4, 2, 0])

In [9]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [10]:
train_x

array([[1.   , 0.908, 0.475, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.104, 0.224, ..., 0.   , 0.   , 0.   ],
       [1.   , 0.786, 0.467, ..., 0.   , 0.   , 0.   ],
       ...,
       [1.   , 0.654, 0.386, ..., 0.   , 0.   , 0.   ],
       [0.966, 0.924, 0.87 , ..., 0.   , 0.   , 0.   ],
       [0.969, 0.592, 0.578, ..., 0.   , 0.   , 0.   ]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

In [12]:
train_x.shape, train_y.shape

((6318, 187, 1), (6318,))

In [13]:
train_x

array([[[1.   ],
        [0.908],
        [0.475],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]],

       [[0.   ],
        [0.104],
        [0.224],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]],

       [[1.   ],
        [0.786],
        [0.467],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]],

       ...,

       [[1.   ],
        [0.654],
        [0.386],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]],

       [[0.966],
        [0.924],
        [0.87 ],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]],

       [[0.969],
        [0.592],
        [0.578],
        ...,
        [0.   ],
        [0.   ],
        [0.   ]]], dtype=float32)

## Baseline Accuracy

In [14]:
data['Target'].value_counts()/len(data)

0.0    0.581875
4.0    0.178152
2.0    0.160425
1.0    0.061600
3.0    0.017948
Name: Target, dtype: float64

# LSTM Model

### Make sure to add the masking layer to the model!

In [15]:
n_steps = 187
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(1, activation='sigmoid' , input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='softmax')
])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [20]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20


InvalidArgumentError:  Received a label value of 4 which is outside the valid range of [0, 1).  Label values: 2 0 0 0 0 2 4 0 4 0 0 1 2 4 0 2 0 2 0 0 2 0 0 0 4 2 0 0 0 2 0 4
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-20-3cbec7d361a6>:8) ]] [Op:__inference_train_function_5055]

Function call stack:
train_function


In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# GRU Model

### Make sure to add the masking layer to this model too!

In [ ]:
n_steps = 187
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(2, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(2, return_sequences=True),
    keras.layers.GRU(2, return_sequences=True),
    keras.layers.GRU(1, activation='sigmoid')
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Discussion

Briefly answer the following questions: (2 points) 
1) Which model performs the best (and why)?<br>
2) What is the baseline value?<br>
3) Does the best model perform better than the baseline (and why)?<br>
4) Does the best model exhibit any overfitting; what did you do about it?

1- LSTM with 6.72% and lower loss rate. <br>
2- 20%<br>
3- No, baseline was 20% and model 6.72%<br>
4- no